# 04 - Modeling with Time Lags

## The Problem with Notebook #3

We got R² = 0.36 — not great. Why?

**Industrial processes have inertia.** When an operator changes pH at 2pm, the effect doesn't show up in lab results until 4pm or 6pm. By using same-time inputs to predict outputs, we were asking the wrong question.

## The Fix: Time-Lagged Features

Instead of: `pH(t) → Silica(t)`

We try: `pH(t-1), pH(t-2), pH(t-3), pH(t-4) → Silica(t)`

We don't know the exact lag, so we include multiple lags and let the model figure out which matters most.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load the ORIGINAL data (not preprocessed) - we'll do preprocessing after adding lags
df = pd.read_csv('../data/processed/mining_hourly.csv', parse_dates=['date'])
df = df.set_index('date').sort_index()

print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Time range: {df.index.min()} to {df.index.max()}")

## Step 1: Define Feature Groups

We'll add lags to the **controllable inputs** — these are what operators change and what has delayed effects.

In [ ]:
# Target
target = '% Silica Concentrate'

# Features to drop (outputs or redundant)
drop_cols = ['% Iron Concentrate', '% Silica Feed']

# Uncontrollable - just use current value (ore quality is what it is)
uncontrollable = ['% Iron Feed']

# Controllable - these have delayed effects, add lags
controllable = [
    'Starch Flow', 'Amina Flow', 
    'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
    'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
    'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
    'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
    'Flotation Column 07 Air Flow',
    'Flotation Column 01 Level', 'Flotation Column 02 Level',
    'Flotation Column 03 Level', 'Flotation Column 04 Level',
    'Flotation Column 05 Level', 'Flotation Column 06 Level',
    'Flotation Column 07 Level',
]

print(f"Uncontrollable inputs: {len(uncontrollable)}")
print(f"Controllable inputs: {len(controllable)}")

## Step 2: Create Time-Lagged Features

For each controllable input, we add:
- **Lag 1-4**: Value from 1, 2, 3, 4 hours ago
- **Rolling mean 4h**: Average over last 4 hours
- **Rolling std 4h**: Variability over last 4 hours (process stability)

In [ ]:
def create_lagged_features(df, columns, max_lag=4):
    """
    Create lagged features for specified columns.
    
    For each column, creates:
    - Lag 1 to max_lag (values from previous hours)
    - Rolling mean over max_lag hours
    - Rolling std over max_lag hours
    """
    df_new = df.copy()
    
    for col in columns:
        # Lagged values
        for lag in range(1, max_lag + 1):
            df_new[f'{col}_lag{lag}'] = df[col].shift(lag)
        
        # Rolling statistics (looking backward)
        df_new[f'{col}_roll_mean_{max_lag}h'] = df[col].shift(1).rolling(window=max_lag).mean()
        df_new[f'{col}_roll_std_{max_lag}h'] = df[col].shift(1).rolling(window=max_lag).std()
    
    return df_new

# Create lagged features for controllable inputs
df_lagged = create_lagged_features(df, controllable, max_lag=4)

print(f"Original columns: {len(df.columns)}")
print(f"After adding lags: {len(df_lagged.columns)}")
print(f"\nNew features per controllable input: 4 lags + 2 rolling = 6")
print(f"Total new features: {len(controllable)} × 6 = {len(controllable) * 6}")

In [ ]:
# Drop rows with NaN (first 4 rows won't have lag data)
rows_before = len(df_lagged)
df_lagged = df_lagged.dropna()
rows_after = len(df_lagged)

print(f"Dropped {rows_before - rows_after} rows (needed for lag calculation)")
print(f"Remaining: {rows_after} rows")

## Step 3: Prepare Features for Modeling

Now we:
1. Drop redundant columns
2. Log-transform Starch Flow (and its lagged versions)
3. Standardize all features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Drop outputs/redundant
df_model = df_lagged.drop(columns=drop_cols)

# Log-transform Starch Flow columns (original + all lagged versions)
starch_cols = [c for c in df_model.columns if 'Starch Flow' in c]
for col in starch_cols:
    df_model[col] = np.log1p(df_model[col])

print(f"Log-transformed {len(starch_cols)} Starch Flow columns")

# Define feature columns (everything except target)
feature_cols = [c for c in df_model.columns if c != target]

print(f"\nTotal features: {len(feature_cols)}")

In [ ]:
# Standardize features
scaler = StandardScaler()
df_model[feature_cols] = scaler.fit_transform(df_model[feature_cols])

# Create X and y
X = df_model[feature_cols]
y = df_model[target]

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeature stats (should be mean≈0, std≈1):")
print(X.describe().T[['mean', 'std']].head(5).round(3))

## Step 4: Time-Based Train/Test Split

**Important:** For time series, we should NOT use random split.

Why? If we randomly mix data, the model might see "future" data during training, which is cheating.

Instead: Train on first 80% of time, test on last 20%.

In [ ]:
# Time-based split (no shuffling!)
split_idx = int(len(X) * 0.8)

X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]
y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

print(f"Training set: {len(X_train)} rows ({X_train.index.min()} to {X_train.index.max()})")
print(f"Test set:     {len(X_test)} rows ({X_test.index.min()} to {X_test.index.max()})")

## Step 5: Train with FLAML (Longer Budget)

We give it 5 minutes this time — more features need more exploration time.

In [ ]:
from flaml import AutoML

automl = AutoML()

automl.fit(
    X_train, y_train,
    task='regression',
    metric='r2',
    time_budget=300,  # 5 minutes
    verbose=1,
    seed=42,
)

print(f"\nBest model: {automl.best_estimator}")

## Step 6: Evaluate

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

y_pred = automl.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("=" * 50)
print("TEST SET PERFORMANCE (with time lags)")
print("=" * 50)
print(f"\nR² Score:  {r2:.4f}")
print(f"RMSE:      {rmse:.4f}")
print(f"MAE:       {mae:.4f}")

print(f"\n--- Comparison with Notebook #3 ---")
print(f"Without lags: R² = 0.359")
print(f"With lags:    R² = {r2:.3f}")
print(f"Improvement:  {(r2 - 0.359) / 0.359 * 100:+.1f}%")

In [ ]:
# Actual vs Predicted plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.scatter(y_test, y_pred, alpha=0.5, s=20)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax.set_xlabel('Actual % Silica')
ax.set_ylabel('Predicted % Silica')
ax.set_title(f'Actual vs Predicted (R² = {r2:.3f})')

ax = axes[1]
residuals = y_test - y_pred
ax.scatter(y_pred, residuals, alpha=0.5, s=20)
ax.axhline(y=0, color='r', linestyle='--', lw=2)
ax.set_xlabel('Predicted % Silica')
ax.set_ylabel('Residual')
ax.set_title('Residual Plot')

plt.tight_layout()
plt.savefig('../data/processed/model_03_lagged_predictions.png', dpi=150)
plt.show()

## Step 7: Which Lags Matter Most?

Feature importance tells us which time lags the model found most useful.

In [ ]:
model = automl.model.estimator

if hasattr(model, 'feature_importances_'):
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Top 20 Most Important Features:")
    print(importance.head(20).to_string(index=False))
else:
    importance = None
    print("Feature importance not available")

In [ ]:
# Analyze which lag times are most important
if importance is not None:
    # Extract lag info from feature names
    def get_lag_type(name):
        if '_lag1' in name:
            return 'lag1 (1h ago)'
        elif '_lag2' in name:
            return 'lag2 (2h ago)'
        elif '_lag3' in name:
            return 'lag3 (3h ago)'
        elif '_lag4' in name:
            return 'lag4 (4h ago)'
        elif '_roll_mean' in name:
            return 'rolling mean'
        elif '_roll_std' in name:
            return 'rolling std'
        else:
            return 'current (t=0)'
    
    importance['lag_type'] = importance['feature'].apply(get_lag_type)
    
    lag_importance = importance.groupby('lag_type')['importance'].sum().sort_values(ascending=False)
    
    print("\nImportance by Lag Type:")
    print("="*40)
    for lag, imp in lag_importance.items():
        print(f"  {lag}: {imp:.0f}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(8, 5))
    lag_importance.plot(kind='bar', ax=ax, color='steelblue')
    ax.set_ylabel('Total Importance')
    ax.set_title('Which Time Lags Matter Most?')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('../data/processed/model_04_lag_importance.png', dpi=150)
    plt.show()

## Summary

### What We Changed:
1. Added lagged features (1-4 hours) for controllable inputs
2. Added rolling mean and std (process stability)
3. Used time-based split instead of random split
4. Increased FLAML budget to 5 minutes

### Key Insight:
The lag analysis tells us the **process response time** — how long after a change do we see the effect in output quality.

### For Operators:
If lag2 or lag3 features are most important, it means changes take 2-3 hours to show up in lab results. This affects how they should:
- Time their adjustments
- Interpret feedback
- Avoid over-correcting